In [1]:
from datasets import load_dataset
import data_loading as data_loading
import pickle

/mount/studenten-temp1/users/godberja/.cslp_proj_env/lib64/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/mount/studenten-temp1/users/godberja/GermanSentiment/data/ids.pkl', 'rb') as file:
    ids = pickle.load(file)

In [3]:
train_ids = ids["de_train_small"]
dev_ids = ids["de_dev_small"]
test_ids = ids["de_test"] 
train_large_ids = ids["de_train_large"]

In [4]:
dataset = load_dataset("Brand24/mms",cache_dir="/mount/studenten-temp1/users/godberja/HuggingfaceCache")
train_data = data_loading.load_own_dataset(dataset,train_ids,None)

In [8]:
train_data_large = data_loading.load_own_dataset(dataset,train_ids,None)

In [6]:
def get_label_distribution(datasplit,split:str):
    neg = 0
    neut = 0
    pos = 0
    for sample in datasplit:
        if sample["label"] == "0":
            neg += 1
        elif sample["label"] == "1":
            neut += 1
        else:
            pos += 1
    all = neg + neut + pos
    return split+": "+str(round(neg/(all)*100))+ "%\t["+str(neg)+"] negative instances\t\t" +str(round(neut/(all)*100))+ "%\t["+str(neut)+"] neutral instances\t\t" +str(round(pos/(all)*100))+ "%\t["+str(pos)+"] positive instances\t\t\n"

In [9]:
get_label_distribution(train_data_large,"large")

'large: 0%\t[0] negative instances\t\t0%\t[0] neutral instances\t\t100%\t[600] positive instances\t\t\n'

In [14]:
for sample in train_data_large:
    print(type(sample["label"]))
    print(sample.keys())
    # print(sample["original_dataset"])
    break

<class 'int'>
dict_keys(['text', 'label', 'id'])


In [17]:
with open("/mount/studenten-temp1/users/godberja/GermanSentiment/evaluation/predictions_german_test_0.pkl","rb") as infile:
    predictions = pickle.load(file)

ValueError: peek of closed file

In [10]:
import pickle
with open('/mount/studenten-temp1/users/godberja/GermanSentiment/evaluation/predictions_german_test_0.pkl', 'rb') as file:
    predictions = pickle.load(file)

In [11]:
from datasets import load_dataset
dataset = load_dataset("Brand24/mms",cache_dir="/mount/studenten-temp1/users/godberja/HuggingfaceCache")

In [15]:
import statistics
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np


In [13]:
def analyze_quantitatively(predictions, dataset, print=False, path_to_save_evaluation_to=None):

    # investigate general performance of model
    preds = [datapoint["pred"] for datapoint in predictions]
    golds = [datapoint["gold"] for datapoint in predictions]
    accuracy = "Accuracy: "+str(metrics.accuracy_score(golds, preds))
    f_score = "F1-score: "+str(metrics.f1_score(golds, preds, average="weighted"))

    # investigate confidence of model
    label_probs = [max(datapoint["prob"]) for datapoint in predictions]
    avg_prob = round(sum(label_probs) / len(label_probs),3)
    min_prob = min(label_probs)
    max_prob = max(label_probs)
    median_prob = round(statistics.median(label_probs),3)
    std_prob = round(statistics.stdev(label_probs),3)
    confidence = "Confidence of model: "+str(avg_prob)+"[avg]\t"+str(std_prob)+"[std dev]\t"+str(min_prob)+"[min]\t"+str(max_prob)+"[max]\t"+str(median_prob)+"[median]"

    # generate a confusion matrix 
    conf_matrix = np.array2string(metrics.confusion_matrix(golds,preds,labels=["0","1","2"]))
    confusion = "Confusion matrix: \n"+str(conf_matrix)
    ConfusionMatrixDisplay.from_predictions(golds,preds)
    plt.title(str("Confusion Matrix"))
    if path_to_save_evaluation_to:
        plt.savefig(str(path_to_save_evaluation_to+"_confusionMatrix.jpg")) 
    plt.close()

    # look at difference across "genre"
    ids = [datapoint["id"] for datapoint in predictions]
    sources = [dataset["train"][data_id]["original_dataset"] for data_id in ids]
    twitter_preds = []
    twitter_golds = []
    review_preds = []
    review_golds = []
    for i, src in enumerate(sources):
        if "amazon" in src:
            review_preds += [predictions[i]["pred"]]
            review_golds += [predictions[i]["gold"]]
        else: # "twitter" in src:
            twitter_preds += [predictions[i]["pred"]]
            twitter_golds += [predictions[i]["gold"]]
    twitter_accuracy = "Accuracy on twitter data only: "+str(metrics.accuracy_score(twitter_golds, twitter_preds))
    review_accuracy = "Accuracy on review data only: "+str(metrics.accuracy_score(review_golds, review_preds))  
    conf_matrix_twitter = np.array2string(metrics.confusion_matrix(twitter_golds,twitter_preds,labels=["0","1","2"]))
    confusion_twitter = "Confusion matrix of twitter data: \n"+str(conf_matrix_twitter)
    plt.clf()
    ConfusionMatrixDisplay.from_predictions(twitter_golds,twitter_preds)
    plt.title(str("Confusion Matrix Twitter data"))
    if path_to_save_evaluation_to:
        plt.savefig(str(path_to_save_evaluation_to+"_confusionMatrix_twitter.jpg")) 
    plt.close()
    conf_matrix_review = np.array2string(metrics.confusion_matrix(review_golds,review_preds,labels=["0","1","2"]))
    confusion_review = "Confusion matrix of review data: \n"+str(conf_matrix_review)
    plt.clf()
    ConfusionMatrixDisplay.from_predictions(review_golds,review_preds)
    plt.title(str("Confusion Matrix Review data"))
    if path_to_save_evaluation_to:
        plt.savefig(str(path_to_save_evaluation_to+"_confusionMatrix_review.jpg")) 
    plt.close()

    # analyze the model performance with the built-in classification_report function
    report = "Built-in analysis: \n"+str(metrics.classification_report(y_true=golds,y_pred=preds))+"\n"

    if print:
        print(accuracy)
        print(f_score)
        print(confidence)
        print(confusion)
        print(report)
        print("\nLet's look at differences between 'genres':")
        print(twitter_accuracy)
        print(review_accuracy)
        print(confusion_twitter)
        print(confusion_review)
    
    if path_to_save_evaluation_to:
        



In [19]:
x = None
if x:
    print("hello")

In [14]:
analyze_quantitatively(predictions,dataset,"/mount/studenten-temp1/users/godberja/GermanSentiment/evaluation/model0")

(0.3333333333333333, 0.16666666666666666)

In [16]:
print(predictions[0])

{'id': 1239901, 'text': 'Anhänger war nach 6 Tagen kaputt', 'gold': '1', 'pred': '0', 'prob': [0.338, 0.334, 0.327]}


In [17]:
ids = [datapoint["id"] for datapoint in predictions]
sources = [dataset["train"][data_id]["original_dataset"] for data_id in ids]   

In [18]:
print(set(sources))

{'de_twitter_sentiment', 'de_multilan_amazon'}


In [6]:
dataset["train"][1239901]

{'_id': 1239901,
 'text': 'Anhänger war nach 6 Tagen kaputt',
 'label': 1,
 'original_dataset': 'de_multilan_amazon',
 'domain': 'reviews',
 'language': 'de',
 'Family': 'Indo-European',
 'Genus': 'Germanic',
 'Definite articles': 'definite word distinct from demonstrative',
 'Indefinite articles': 'indefinite word same as one',
 'Number of cases': '4',
 'Order of subject, object, verb': 'no dominant order',
 'Negative morphemes': 'negative particle',
 'Polar questions': 'interrogative word order',
 'Position of negative word wrt SOV': 'more than one position',
 'Prefixing vs suffixing': 'strongly suffixing',
 'Coding of nominal plurality': 'plural suffix',
 'Grammatical genders': 'masculine, feminine, neuter',
 'cleanlab_self_confidence': 0.0547509491443634}